In [ ]:
import os
import yaml
import dotenv

dotenv.load_dotenv()

In [ ]:
from langchain_community.agent_toolkits.openapi.spec import reduce_openapi_spec
from langchain_community.agent_toolkits.openapi import planner
# from langchain_openai import ChatOpenAI
from langchain.requests import RequestsWrapper
import tiktoken

In [ ]:
from langchain_community.chat_models.litellm import ChatLiteLLM

In [ ]:
boardroom_api_key = os.getenv("BOARDROOM_API_KEY")
boardroom_api_key

## Loading OpenAPI Spec

In [ ]:
with open("../boardroom/boardroom_openapi.yaml") as f:
    raw_boardroom_api_spec = yaml.load(f, Loader=yaml.Loader)
boardroom_api_spec = reduce_openapi_spec(raw_boardroom_api_spec)

In [ ]:
raw_boardroom_api_spec['security'][0]['api-key'] = [boardroom_api_key]

In [ ]:
headers = {"Accept": "application/json"} # add `Authorization` header if an access token is required
requests_wrapper = RequestsWrapper(headers=headers)

In [ ]:
# How big is the spec?
endpoints = [
    (route, operation)
    for route, operations in raw_boardroom_api_spec["paths"].items()
    for operation in operations
    if operation in ["get", "post"]
]
len(endpoints)

In [ ]:
enc = tiktoken.encoding_for_model("gpt-4")


def count_tokens(s):
    return len(enc.encode(s))


count_tokens(yaml.dump(raw_boardroom_api_spec))

## Loading Boardroom Agent

In [ ]:
# NOTE: set allow_dangerous_requests manually for security concern https://python.langchain.com/docs/security
ALLOW_DANGEROUS_REQUEST=True

In [ ]:
from typing import (Dict, Any)

class MyChatLiteLLM(ChatLiteLLM):
    @property
    def _default_params(self) -> Dict[str, Any]:
        """Get the default parameters for calling OpenAI API."""
        set_model_value = self.model
        if self.model_name is not None:
            set_model_value = self.model_name
        return {
            "model": set_model_value,
            "force_timeout": self.request_timeout,
            "max_tokens": self.max_tokens,
            "stream": self.streaming,
            # "n": self.n, # NOTICE: needed to override function to avoid sending this unknown parameter on the current version of RAG API Server
            "temperature": self.temperature,
            "custom_llm_provider": self.custom_llm_provider,
            **self.model_kwargs,
        }

In [ ]:
# llm = ChatOpenAI(model_name="gpt-4", temperature=0.0)
llm = MyChatLiteLLM(
    # api_base="https://0xd0275a4f8ef0d4b2dc0a3baf3d405a63bc647340.gaianet.network/v1",
    api_base="https://llama.us.gaianet.network/v1",
    openai_api_key="",
    # model_name="Llama-3-8B-Instruct",
    model_name="llama",
    custom_llm_provider="openai",
    temperature=0.5,
    max_tokens=8000, # based on context size in node config
    verbose=True
)

In [ ]:
boardroom_agent = planner.create_openapi_agent(
    boardroom_api_spec,
    requests_wrapper,
    llm,
    allow_dangerous_requests=ALLOW_DANGEROUS_REQUEST,
    # agent_executor_kwargs={'handle_parsing_errors': True}
)

In [ ]:
user_query = (
    f"""
        I Need to get a full list of proposals from 'aave' protocool order by date in descending order.
        Use '{boardroom_api_key}' as 'apiKey' url path parameter on every request made to the API
    """
)
user_query

In [ ]:
import litellm
litellm.set_verbose=True

os.environ['LITELLM_LOG'] = 'DEBUG'

In [ ]:
boardroom_agent.invoke(user_query)